#### Required Installs

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install soundfile

In [ ]:
!pip install protobuf

In [ ]:
!pip install librosa

In [ ]:
!apt update
!apt install -y ffmpeg

In [ ]:
!pip install ffmpeg

In [ ]:
!pip install torchaudio

In [ ]:
!pip install gradio

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install evaluate

In [44]:
!pip install -U wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.0 MB/s eta 0:00:00


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


  Attempting uninstall: wandb
    Found existing installation: wandb 0.13.4
    Uninstalling wandb-0.13.4:
      Successfully uninstalled wandb-0.13.4


In [ ]:
!apt-get install git-lfs

In [ ]:
!git lfs install

In [40]:
# have to login to access speech-commands dataset
from huggingface_hub import login
login()

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: ptah23. Use `wandb login --relogin` to force relogin


True

# Build a music genre classifier

## Pre-trained models and datasets for audio classification

### Keyword spotting

#### Minds-14

In [3]:
from datasets import load_dataset
minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset minds14 downloaded and prepared to /root/.cache/huggingface/datasets/PolyAI___minds14/en-AU/1.0.0/65c7e0f3be79e18a6ffaf879a083daf706312d421ac90d25718459cbf3c42696. Subsequent calls will reuse this data.


In [4]:
from transformers import pipeline
classifier = pipeline("audio-classification", model="anton-l/xtreme_s_xlsr_300m_minds14")

In [5]:
classifier(minds[0]["path"])

[{'score': 0.9623648524284363, 'label': 'pay_bill'},
 {'score': 0.02867838554084301, 'label': 'freeze'},
 {'score': 0.0034296163357794285, 'label': 'card_issues'},
 {'score': 0.0020604876335710287, 'label': 'abroad'},
 {'score': 0.0008625672780908644, 'label': 'high_value_payment'}]

In [6]:
import IPython
IPython.display.Audio(minds[0]["path"])

#### Speech commands

In [7]:
speech_commands = load_dataset("speech_commands", "v0.02", split="validation", streaming=True)
sample = next(iter(speech_commands))
sample

{'file': 'backward/0d82fd99_nohash_2.wav',
 'audio': {'path': 'backward/0d82fd99_nohash_2.wav',
  'array': array([-9.15527344e-05,  6.10351562e-05,  6.10351562e-05, ...,
          2.68859863e-02,  1.46179199e-02,  1.67846680e-03]),
  'sampling_rate': 16000},
 'label': 30,
 'is_unknown': True,
 'speaker_id': '0d82fd99',
 'utterance_id': 2}

In [8]:
IPython.display.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [9]:
classifier = pipeline("audio-classification", model="MIT/ast-finetuned-speech-commands-v2")
classifier(sample["audio"])

[{'score': 0.9999892711639404, 'label': 'backward'},
 {'score': 1.7504938796264469e-06, 'label': 'happy'},
 {'score': 6.703033363919531e-07, 'label': 'follow'},
 {'score': 5.805895852972753e-07, 'label': 'stop'},
 {'score': 5.614552378574444e-07, 'label': 'up'}]

### Language identification

#### FLEURS

In [10]:
fleurs = load_dataset("google/fleurs", "all", split="validation", streaming=True)
sample = next(iter(fleurs))

In [11]:
sample

{'id': 1326,
 'num_samples': 218880,
 'path': None,
 'audio': {'path': 'dev/1015649832594091373.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ...,  0.0002867 ,
         -0.00020713, -0.0001356 ]),
  'sampling_rate': 16000},
 'transcription': 'in 1989 het hy vir brooks en groening gehelp om the simpsons te skep en hy was verantwoordelik om die eerste skrywerspan vir die program te huur',
 'raw_transcription': 'In 1989 het hy vir Brooks en Groening gehelp om The Simpsons te skep en hy was verantwoordelik om die eerste skrywerspan vir die program te huur.',
 'gender': 0,
 'lang_id': 0,
 'language': 'Afrikaans',
 'lang_group_id': 3}

In [12]:
IPython.display.Audio(data=sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [13]:
classifier = pipeline("audio-classification", model="sanchit-gandhi/whisper-medium-fleurs-lang-id")

In [14]:
classifier(sample["audio"])

[{'score': 0.9999330043792725, 'label': 'Afrikaans'},
 {'score': 7.093023668858223e-06, 'label': 'Northern-Sotho'},
 {'score': 4.269153578206897e-06, 'label': 'Icelandic'},
 {'score': 3.266117346356623e-06, 'label': 'Danish'},
 {'score': 3.2580724109720904e-06, 'label': 'Cantonese Chinese'}]

### Zero-shot audio classification

In [15]:
dataset = load_dataset("ashraq/esc50", split="train", streaming=True)
sample= next(iter(dataset))
audio_sample = sample["audio"]["array"]
sample

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Using custom data configuration ashraq--esc50-1000c3b73cc1500f


{'filename': '1-100032-A-0.wav',
 'fold': 1,
 'target': 0,
 'category': 'dog',
 'esc10': True,
 'src_file': 100032,
 'take': 'A',
 'audio': {'path': None,
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 44100}}

In [16]:
IPython.display.Audio(data=audio_sample, rate=44_100)

In [17]:
candidate_labels=["Sound of a dog", "Sound of vacuum cleaner"]

In [18]:
classifier = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-unfused")
classifier(audio_sample, candidate_labels=candidate_labels)

[{'score': 0.9997242093086243, 'label': 'Sound of a dog'},
 {'score': 0.00027583030168898404, 'label': 'Sound of vacuum cleaner'}]

## Fine-tuning a model for music classification

### The Dataset

In [19]:
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Using custom data configuration all


Generating train split: 0 examples [00:00, ? examples/s]

Dataset gtzan downloaded and prepared to /root/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/8bd0e23c2d9b2be30d36bc6834319772dff22a3bd28527996612386cef003910. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [20]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [21]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229], dtype=float32),
  'sampling_rate': 22050},
 'genre': 7}

In [22]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

In [23]:
# import gradio as gr
# def generate_audio():
#     example = gtzan["train"].shuffle()[0]
#     audio = example["audio"]
#     return (
#         audio["sampling_rate"],
#         audio["array"],
#     ), id2label_fn(example["genre"])


# with gr.Blocks() as demo:
#     with gr.Column():
#         for _ in range(4):
#             audio, label = generate_audio()
#             output = gr.Audio(audio, label=label)

# demo.launch(debug=True, share=True)

### Picking a pretrained model for audio classification

In [24]:
from transformers import AutoFeatureExtractor
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id, do_normalize=True, return_attention_mask=True)

In [25]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [26]:
from datasets import Audio
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [27]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427], dtype=float32),
  'sampling_rate': 16000},
 'genre': 7}

In [28]:
import numpy as np
sample = gtzan["train"][0]["audio"]
print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


In [29]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])
print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)
                                                                 

inputs keys: ['input_values', 'attention_mask']
Mean: -7.45e-09, Variance: 1.0


In [30]:
max_duration = 30.0
def preprocess_function(examples):
    audio_arrays=[x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=int(feature_extractor.sampling_rate * max_duration), truncation=True, return_attention_mask=True)
    return inputs

In [31]:
gtzan_encoded = gtzan.map( preprocess_function, remove_columns=["audio", "file"], batched=True, num_proc=1)
gtzan_encoded

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [32]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [33]:
id2label = { str(i): id2label_fn(i) for i in range(len(gtzan_encoded["train"].features["label"].names))}
label2id = {v: k for k,v in id2label.items()}
id2label["7"]

'pop'

In [34]:
from transformers import AutoModelForAudioClassification
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(model_id, num_labels=num_labels, label2id=label2id, id2label=id2label)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['projector.weight', 'projector.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
from transformers import TrainingArguments
model_name = model_id.split("/")[-1]
batch_size=8
gradient_accumulation_steps=1
num_train_epochs=20
%env WANDB_PROJECT=huggingface-audio-course
training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    report_to="wandb",

)

env: WANDB_PROJECT=huggingface-audio-course


In [52]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)
                       

In [50]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)
trainer.train()
#trainer.train(resume_from_checkpoint=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/notebooks/distilhubert-finetuned-gtzan is already a clone of https://huggingface.co/ptah23/distilhubert-finetuned-gtzan. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
6,0.372500,0.633845,0.800000
7,0.449800,0.553892,0.850000
8,0.223200,0.540494,0.830000
9,0.245600,0.525873,0.850000
10,0.177700,0.555530,0.840000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=1130, training_loss=0.13275848094340975, metrics={'train_runtime': 2220.6703, 'train_samples_per_second': 4.048, 'train_steps_per_second': 0.509, 'total_flos': 6.133988274624e+17, 'train_loss': 0.13275848094340975, 'epoch': 10.0})

In [53]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}
trainer.push_to_hub(**kwargs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/ptah23/distilhubert-finetuned-gtzan
   2dc2d82..4388e69  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/ptah23/distilhubert-finetuned-gtzan
   4388e69..6ee0123  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/ptah23/distilhubert-finetuned-gtzan/commit/4388e69e1ffa525ae9e5f43b04625da4bc806596'